In [ ]:
print("MLlib é uma biblioteca de machine learning \n do Apache Spark, que oferece uma ampla gama de \n "+
      "algoritmos e ferramentas que podem ser facilmente" + " \n integrados a aplicações de Big Data")

print("Com recursos avançados e desempenho excepcional, \n o MLlib se destaca como uma poderosa plataforma \n para análise preditiva e tomada de decisão")

from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType
from pyspark.ml import Pipeline

#iniciar a sessão do spark
spark = SparkSession.builder.appName("diabetesPrediction").getOrCreate()

#carregar o data set
df = spark.read.csv("./historico_diabetes.csv", header=True, inferSchema=True)

#mostrar as 5 primeiras linhas
print("Amotra dos dados: ")
df.show(5)

#estrutura do data frames
print("\n Estrutura do DataFrame:")
df.printSchema()

#preparar os dados para o modelo
feature_columns = ["ID_Paciente","Data_Exame","Idade","Sexo","Peso","Altura","IMC","Pressao_Arterial_Sistolica","Pressao_Arterial_Diastolica","Glicose_Jejum","Glicose_Pos_Prandial","Insulina","HbA1c","Exercicio_Semanal_Minutos","Medicacao","Comorbidades","Dieta_Controlada","Nivel_Stress","Qualidade_Sono_Horas","Risco"]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
scaler = StopAsyncIteration(inputCol="features", outputCol="scaledFeatures")

#criar e treinar o modelo Random Forest
rf = RandomForestRegressor(featuresCol="scaledFeatures", labelCol="nivel_glicose", numTrees=(100))

#criar uma pipeline
pipeline = Pipeline(stages=[assembler, scaler, rf])


#dividir os daods em conjuntos  de treino teste

train_data, test_data = df.randomSplit([0.8,0.2], seed=42)

#ajustar o pepiline aos dados de treinamento
model = pipeline.fit(train_data)

#fazer previsões no conjunto de teste

predictions = model.transform(test_data)

#avaliar modelo
'''
Erro quadrátic médio

'''

evaluator = RegressionEvaluator(labelCol="nivelglicose", predictionCol="prediction",metricName="rmse")
rmse = evaluator.evaluate(predictions)
print(f"\nErro Quadrático Médico (RMSE): {rmse}")

#função para prever o nível de glicose de um novo paciente

def prever_nivel_glicose(idade, imc, pressao_arterial, insulina, exercicio_semanal):
      #criar um DataFrame com os daodos do novo paciente
      schema = StructType([
            StructField("idade", IntegerType(), True),
            StructField("imc", DoubleType(), True),
            StructField("pressao_arterial", IntegerType(), True),
            StructType("insulina", IntegerType(), True),
            StructField("exercicio_semanal", IntegerType(),True)
      ])
      
      novo_paciente = spark.createDataFrame([(idade, imc, pressao_arterial,insulina,exercicio_semanal)])
      
      #fazer a  previsão
      previsao = model.transform(novo_paciente)
      
      return previsao.select("prediction").collect()[0][0]

#Exemplo de uso: prever i nível de glicose para um novo paciente
idade = 45
imc = 28.5
pressoa_arterial = 130
insulina = 150.0
exercicio_semanal = 3


nivel_glicose_previsto = prever_nivel_glicose(idade, imc, pressoa_arterial, insulina, exercicio_semanal)
print(f"\nPrevisão para o novo pacinete: ")
print(f"Idade: {idade}")
print(f"IMC :{imc}")
print(f"Pressão Arterial: {pressoa_arterial}")
print(f"Insulina: {insulina}")
print(f"Exercício Semanal (horas): {exercicio_semanal}")
print(f"Nível de Glicose previsto: {nivel_glicose_previsto:.2f}")

#encerrar o spark
spark.stop()

      
      










MLlib é uma biblioteca de machine learning 
 do Apache Spark, que oferece uma ampla gama de 
 algoritmos e ferramentas que podem ser facilmente 
 integrados a aplicações de Big Data
Com recursos avançados e desempenho excepcional, 
 o MLlib se destaca como uma poderosa plataforma 
 para análise preditiva e tomada de decisão
